In [1]:
import json
import web3

#from web3 import Web3
from web3 import Web3, HTTPProvider
from solc import compile_source
from web3.contract import ConciseContract

import pandas as pd

data = pd.read_csv("Llaves.csv") 

public_keys = data['Llave Publica']
private_keys = data['Llave Privada']

w3 = Web3(HTTPProvider("https://rinkeby.infura.io/v3/4c0ec7f1412a489d91e1934c66ebf5b1"))

## Transfer Ether

In [32]:
acct = w3.eth.account.privateKeyToAccount(private_keys[0])
totalSupply = w3.eth.getBalance(public_keys[0])
print('Total Supply = '+str(totalSupply))
balance = 1
for address in public_keys[1:]:
    
    signed_txn = w3.eth.account.signTransaction({
        'from': acct.address,
        'nonce': w3.eth.getTransactionCount(acct.address),
        'gas': 3000000,
        'to':address,
        'gasPrice': w3.toWei('1.4', 'gwei'),
        'value':w3.toWei(1,'ether')},
        private_keys[0])
    
    tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    # Wait for the transaction to be mined, and get the transaction receipt
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

Total Supply = 23643331852400000000


## Coops

In [2]:
# Solidity source code
Coops_source_code = '''
pragma solidity ^0.4.24;

//import "./Owned.sol";
//import "./CrowdFunding.sol";

// ----------------------------------------------------------------------------
// 'Coops' token contract
//
// Deployed to : 0xdD870fA1b7C4700F2BD7f44238821C26f7392148
// Symbol      : Coops
// Name        : Coops Token
// Total supply: 1000000000
// Decimals    : 0
//
// Enjoy.
//
// (c) by Moritz Neto with BokkyPooBah / Bok Consulting Pty Ltd Au 2017. The MIT Licence.
// ----------------------------------------------------------------------------


// ----------------------------------------------------------------------------
// Safe maths
// ----------------------------------------------------------------------------
contract SafeMath {
    function safeAdd(uint a, uint b) public pure returns (uint c) {
        c = a + b;
        require(c >= a);
    }
    function safeSub(uint a, uint b) public pure returns (uint c) {
        require(b <= a);
        c = a - b;
    }
    function safeMul(uint a, uint b) public pure returns (uint c) {
        c = a * b;
        require(a == 0 || c / a == b);
    }
    function safeDiv(uint a, uint b) public pure returns (uint c) {
        require(b > 0);
        c = a / b;
    }
}


// ----------------------------------------------------------------------------
// ERC Token Standard #20 Interface
// https://github.com/ethereum/EIPs/blob/master/EIPS/eip-20-token-standard.md
// ----------------------------------------------------------------------------
contract ERC20Interface {
    function totalSupply() public constant returns (uint);
    function balanceOf(address tokenOwner) public constant returns (uint balance);
    function allowance(address tokenOwner, address spender) public constant returns (uint remaining);
    function transfer(address to, uint tokens) public returns (bool success);
    function approve(address spender, uint tokens) public returns (bool success);
    function transferFrom(address from, address to, uint tokens) public returns (bool success);

    event Transfer(address indexed from, address indexed to, uint tokens);
    event Approval(address indexed tokenOwner, address indexed spender, uint tokens);
}


// ----------------------------------------------------------------------------
// Contract function to receive approval and execute function in one call
//
// Borrowed from MiniMeToken
// ----------------------------------------------------------------------------
contract ApproveAndCallFallBack {
    function receiveApproval(address from, uint256 tokens, address token, bytes data) public;
}

// ----------------------------------------------------------------------------
// Owned contract
// ----------------------------------------------------------------------------
contract Owned {
    address public owner;
    address public newOwner;

    event OwnershipTransferred(address indexed _from, address indexed _to);

    constructor() public {
        owner = msg.sender;
    }

    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }

    function transferOwnership(address _newOwner) public onlyOwner {
        newOwner = _newOwner;
    }
    function acceptOwnership() public {
        require(msg.sender == newOwner);
        emit OwnershipTransferred(owner, newOwner);
        owner = newOwner;
        newOwner = address(0);
    }
}

contract CrowdFunding is Owned{
    
    address public beneficiary;
    uint public fundingGoal;
    uint public amountRaised;
    uint public deadline;
    uint numFunders = 0;
    mapping(address => uint256) public balanceOf;
    //mapping(uint => address) Funders;
    //mapping(uint => uint) Funds;
    address[] Funders;
    uint[] Funds;
    bool fundingGoalReached = false;
    bool crowdsaleClosed = true;

    event GoalReached(address recipient, uint totalAmountRaised);
    event FundTransfer(address backer, uint amount, bool isContribution);
    
    address Ccontract = this;
    Coops C = Coops(Ccontract);

    function receiveFund
    (address _sender,
    uint256 _tokens,
    Coops _tokenContract)
    public{
      require(_tokenContract == C);
      require(!crowdsaleClosed);
      if(amountRaised > fundingGoal) revert();
      require(C.transferFrom(_sender, this, _tokens));
      balanceOf[_sender] += _tokens;
      amountRaised += _tokens;
      //Funders[numFunders] = _sender;
      //Funds[numFunders] = _tokens;
      Funders.push(_sender);
      Funds.push(_tokens);
      numFunders += 1;
      emit FundTransfer(msg.sender, _tokens, true);
    }

    modifier afterDeadline() { if (now >= deadline) _; }

}




// ----------------------------------------------------------------------------
// ERC20 Token, with the addition of symbol, name and decimals and assisted
// token transfers
// ----------------------------------------------------------------------------



contract Coops is ERC20Interface, Owned, SafeMath {
    
    struct Project{
        address owner;
        address[] funders;
        uint[] amounts;
        uint totalPrice;
        uint ID;
    }
    
    
    string public symbol;
    string public  name;
    
    
    uint8 public decimals;
    uint public _totalSupply;
    
    uint public numProjects = 0;
    uint public kwh_price = 100;

    mapping(address => uint) balances;
    mapping(address => mapping(address => uint)) allowed;
    mapping(address => mapping(address => uint)) private Debts;
    mapping(address => uint) private totalDebts;
    mapping(uint => Project) private Projects; 

    // ------------------------------------------------------------------------
    // Constructor
    // ------------------------------------------------------------------------
    constructor() public {
        symbol = "Coops";
        name = "Coops Token";
        decimals = 0;
        _totalSupply = 1000000000;
        balances[owner] = _totalSupply;
        emit Transfer(address(0), owner, _totalSupply);
    }


    // ------------------------------------------------------------------------
    // Total supply
    // ------------------------------------------------------------------------
    function totalSupply() public constant returns (uint) {
        return _totalSupply  - balances[address(0)];
    }


    // ------------------------------------------------------------------------
    // Get the token balance for account tokenOwner
    // ------------------------------------------------------------------------
    function balanceOf(address tokenOwner) public constant returns (uint balance) {
        return balances[tokenOwner];
    }
    
    function getDebt(address debtor, address creditter) 
    public
    constant
    returns(uint debt)
    {
    return (Debts[debtor][creditter]);
    }
    
    function getTotalDebt(address debtor) 
    public
    constant
    returns(uint debt)
    {
    return (totalDebts[debtor]);
    }


    // ------------------------------------------------------------------------
    // Transfer the balance from token owner's account to to account
    // - Owner's account must have sufficient balance to transfer
    // - 0 value transfers are allowed
    // ------------------------------------------------------------------------
    function transfer(address to, uint tokens) public returns (bool success) {
        balances[msg.sender] = safeSub(balances[msg.sender], tokens);
        balances[to] = safeAdd(balances[to], tokens);
        emit Transfer(msg.sender, to, tokens);
        return true;
    }

    function Pay(address creditter, uint amount) 
    public
    returns(bool success) {
        balances[msg.sender] = safeSub(balances[msg.sender], amount);
        //Debts[debtor][creditter] -= amount;
        Debts[msg.sender][creditter] = safeSub(Debts[msg.sender][creditter],amount);
        //totalDebts[debtor] -= amount;
        totalDebts[msg.sender] = safeSub(totalDebts[msg.sender],amount);
        balances[creditter] = safeAdd(balances[creditter], amount);
        return true;
    } 

    // ------------------------------------------------------------------------
    // Token owner can approve for spender to transferFrom(...) tokens
    // from the token owner's account
    //
    // https://github.com/ethereum/EIPs/blob/master/EIPS/eip-20-token-standard.md
    // recommends that there are no checks for the approval double-spend attack
    // as this should be implemented in user interfaces 
    // ------------------------------------------------------------------------
    function approve(address spender, uint tokens) public returns (bool success) {
        allowed[msg.sender][spender] = tokens;
        emit Approval(msg.sender, spender, tokens);
        return true;
    }
    
    function addDebt(address debtor, address creditter, uint amount) 
    public
    returns(bool success) 
  {
    //Debts[debtor][creditter] += amount;
    Debts[debtor][creditter] = safeAdd(Debts[debtor][creditter], amount);
    //totalDebts[debtor] += amount;
    totalDebts[debtor] = safeAdd(totalDebts[debtor], amount);
    return true;
  }


    // ------------------------------------------------------------------------
    // Transfer tokens from the from account to the to account
    // 
    // The calling account must already have sufficient tokens approve(...)-d
    // for spending from the from account and
    // - From account must have sufficient balance to transfer
    // - Spender must have sufficient allowance to transfer
    // - 0 value transfers are allowed
    // ------------------------------------------------------------------------
    function transferFrom(address from, address to, uint tokens) public returns (bool success) {
        balances[from] = safeSub(balances[from], tokens);
        allowed[from][msg.sender] = safeSub(allowed[from][msg.sender], tokens);
        balances[to] = safeAdd(balances[to], tokens);
        emit Transfer(from, to, tokens);
        return true;
    }

    // ------------------------------------------------------------------------
    // Returns the amount of tokens approved by the owner that can be
    // transferred to the spender's account
    // ------------------------------------------------------------------------
    function allowance(address tokenOwner, address spender) public constant returns (uint remaining) {
        return allowed[tokenOwner][spender];
    }


    // ------------------------------------------------------------------------
    // Token owner can approve for spender to transferFrom(...) tokens
    // from the token owner's account. The spender contract function
    // receiveApproval(...) is then executed
    // ------------------------------------------------------------------------
    /*function approveAndCall(address spender, uint tokens, bytes data) public returns (bool success) {
        allowed[msg.sender][spender] = tokens;
        emit Approval(msg.sender, spender, tokens);
        ApproveAndCallFallBack(spender).receiveApproval(msg.sender, tokens, this, data);
        return true;
    }*/
    function Fund(address spender, uint tokens) public returns (bool success) {
        require(balanceOf(msg.sender)>0);
        require(totalDebts[msg.sender]==0);
        allowed[msg.sender][spender] = tokens;
        emit Approval(msg.sender, spender, tokens);
        //ApproveAndCallFallBack(spender).receiveApproval(msg.sender, tokens, this, data);
        //CrowdFunding F = CrowdFunding(Fcontract)
        CrowdFunding(spender).receiveFund(
            msg.sender, 
            tokens, 
            this);
        return true;
    }


    // ------------------------------------------------------------------------
    // Don't accept ETH
    // ------------------------------------------------------------------------
    function () public payable {
        revert();
    }


    // ------------------------------------------------------------------------
    // Owner can transfer out any accidentally sent ERC20 tokens
    // ------------------------------------------------------------------------
    function transferAnyERC20Token(address tokenAddress, uint tokens) public onlyOwner returns (bool success) {
        return ERC20Interface(tokenAddress).transfer(owner, tokens);
    }
    
    // ------------------------------------------------------------------------
    // Projects
    // ------------------------------------------------------------------------
    function NewProject(
        address owner,
        address[] funders, 
        uint[] amounts, 
        uint totalPrice) 
        public {
        //require(canCreate[msg.sender] == true);
        //require(D.getTotalDebt(owner) == 0);
        Projects[numProjects].owner = owner;
        Projects[numProjects].funders = funders;
        Projects[numProjects].amounts = amounts;
        Projects[numProjects].totalPrice = totalPrice;
        Projects[numProjects].ID = numProjects;
        numProjects += 1;
    }
    
    function Generate(uint kwh, uint ID) 
      public 
      {
        require(Projects[ID].owner == msg.sender);
        uint amount = kwh*kwh_price;
        uint amount_i;
        for (uint i = 0; i < (Projects[ID].funders).length; i++){
          amount_i = amount*(Projects[ID].amounts[i])/(Projects[ID].totalPrice);
          addDebt(msg.sender, (Projects[ID].funders[i]), amount_i);
        }
    }
}
'''

Coops_compiled_sol = compile_source(Coops_source_code,  import_remappings=['=./', '-']) # Compiled source code
Coops_interface = Coops_compiled_sol['<stdin>:Coops']

coops = w3.eth.contract(
    address = '0x91c3010Caca9AC8817600AA2E53a8FAcde8e2bfd',
    abi = Coops_interface['abi'],
)

## Repartir Coops

In [3]:
i_addresses = 5
acct = w3.eth.account.privateKeyToAccount(private_keys[0])
balance = int(coops.call().balanceOf(public_keys[0])/(i_addresses))
for i in range(1,i_addresses+1):
    address = public_keys[i]
    i_balance = int(coops.call().balanceOf(public_keys[i]))
    if i_balance == 0: 
        construct_txn = coops.functions.transfer(address,balance).buildTransaction({
            'from': acct.address,
            'nonce': w3.eth.getTransactionCount(acct.address),
            'gas': 3000000,
            'gasPrice': w3.toWei('1.4', 'gwei')})
        signed = acct.signTransaction(construct_txn)

        tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
        # Wait for the transaction to be mined, and get the transaction receipt
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    print('Balance of '+address+' = '+str(coops.call().balanceOf(address))+ '  Coops')

Balance of 0x6C740120A054cecF209BCaCF62fC38Cc64D98499 = 200000000  Coops
Balance of 0x520597cF4C6f9aed321a39a12529911350c7d0BF = 200000000  Coops
Balance of 0x6c2a6C2F7B3ABf37575a423E72d91C78F72B36E8 = 200000000  Coops
Balance of 0x29b93fdD9D9BE8c80d483aA94CA12496f09006C8 = 200000000  Coops
Balance of 0xF7ab85D5391651B239873fE0Cf26B6B2f0E52358 = 200000000  Coops


## Devolver Coops

In [4]:
for i in range(1,len(public_keys)):
    address = public_keys[i]
    balance = coops.call().balanceOf(address)
    if balance > 0:
        acct = w3.eth.account.privateKeyToAccount(private_keys[i])
        construct_txn = coops.functions.transfer(public_keys[0],balance).buildTransaction({
            'from': acct.address,
            'nonce': w3.eth.getTransactionCount(acct.address),
            'gasPrice': w3.toWei('1.4', 'gwei')})

        signed = acct.signTransaction(construct_txn)

        tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
        # Wait for the transaction to be mined, and get the transaction receipt
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
        
    print('Balance of '+address+' = '+str(coops.call().balanceOf(address))+ '  Coops')



Balance of 0x6C740120A054cecF209BCaCF62fC38Cc64D98499 = 0  Coops
Balance of 0x520597cF4C6f9aed321a39a12529911350c7d0BF = 0  Coops
Balance of 0x6c2a6C2F7B3ABf37575a423E72d91C78F72B36E8 = 0  Coops
Balance of 0x29b93fdD9D9BE8c80d483aA94CA12496f09006C8 = 0  Coops
Balance of 0xF7ab85D5391651B239873fE0Cf26B6B2f0E52358 = 0  Coops
Balance of 0xe19e53113C2901eDec0Ff21a30Bcc28CFEB7c04B = 0  Coops
Balance of 0x2E3CaF2E65021aaA035c8ede24CCb361aC16659A = 0  Coops
Balance of 0x3f274122c33CED256d0F5D46f420ad6b92d4D9E5 = 0  Coops
Balance of 0xc30B42346A4d15AAdfA44cBf885E758dbe110b86 = 0  Coops
Balance of 0xa543D290F7B5FFCc94999935CF8E50f70a29C9bb = 0  Coops
Balance of 0x46D2b966Cf2195145aA3A9028aCDf0A1763E065F = 0  Coops
Balance of 0xAb7F3ccbBEb7d93605bDE9E0589F773FD4F21EC4 = 0  Coops
Balance of 0xbe9dAaA8007a325d21BA5104bCba75085e22b5E0 = 0  Coops
Balance of 0x13079510D41706bc7880d7C20Df1a5AaE3A26A25 = 0  Coops
Balance of 0xE793eF9f5D743F0Bb959144A34de76cD0b8eF6B4 = 0  Coops
Balance of 0xbd10542D544C

## Propietarios

In [5]:
# Solidity source code
Propietarios_source_code = '''

pragma solidity ^0.4.24;

//import "./Owned.sol";

// ----------------------------------------------------------------------------
// Owned contract
// ----------------------------------------------------------------------------

contract Propietarios{

  struct Demand {
    uint maxPrice;
    bool Funding;
    uint index;
  }
  /*address Ccontract = 0x23e1bB98a277fd0e974bafC7bB31b5fBfC8CF358;
  Coops C = Coops(Ccontract);*/
  mapping(address => Demand) private Demands;
  address[] private DemandIndex;
  
  mapping(address => int) private Scores;
  mapping(address => mapping(address => bool)) canRate;

  event LogNewDemand   (address indexed userAddress, uint index, bool Funding, uint maxPrice);
  event LogUpdateDemand(address indexed userAddress, uint index, bool Funding, uint maxPrice);
  event LogDeleteDemand(address indexed userAddress, uint index);
  event LogNewRate(address indexed Rated, int Score);
  
  modifier onlyBy(address _account)
  {
    require(
        msg.sender == _account,
        "Sender not authorized."
    );
    // Do not forget the "_;"! It will
    // be replaced by the actual function
    // body when the modifier is used.
    _;
  }
  
  function isDemand(address userAddress)
    public 
    constant
    returns(bool isIndeed) 
  {
    if(DemandIndex.length == 0) return false;
    return (DemandIndex[Demands[userAddress].index] == userAddress);
  }

  function newDemand(
    bool    Funding, 
    uint    maxPrice) 
    public
    returns(uint index)
  {
    if(isDemand(msg.sender)) revert(); 
    Demands[msg.sender].Funding = Funding;
    Demands[msg.sender].maxPrice   = maxPrice;
    Demands[msg.sender].index     = DemandIndex.push(msg.sender)-1;
    emit 
        LogNewDemand(
        msg.sender, 
        Demands[msg.sender].index, 
        Funding, 
        maxPrice);
    return DemandIndex.length-1;
  }

  function deleteDemand(address userAddress) 
    public
    returns(uint index)
  {
    if(!isDemand(userAddress)) revert(); 
    uint rowToDelete = Demands[userAddress].index;
    address keyToMove = DemandIndex[DemandIndex.length-1];
    DemandIndex[rowToDelete] = keyToMove;
    Demands[keyToMove].index = rowToDelete; 
    DemandIndex.length--;
    emit
    LogDeleteDemand(
        userAddress, 
        rowToDelete);
    emit
    LogUpdateDemand(
        keyToMove, 
        rowToDelete, 
        Demands[keyToMove].Funding, 
        Demands[keyToMove].maxPrice);
    return rowToDelete;
  }
  
  function getDemand(address userAddress)
    public 
    constant
    returns(uint maxPrice)
  {
    if(!isDemand(userAddress)) revert(); 
    return(Demands[userAddress].maxPrice); 
      //Demands[userAddress].index);
  } 

  function getFunding(address userAddress)
    public 
    constant
    returns(bool Funding)
  {
    if(!isDemand(userAddress)) revert(); 
    return(Demands[userAddress].Funding); 
      //Demands[userAddress].index);
  } 
  
  function updateFunding(bool Funding) 
    public
    returns(bool success) 
  {
    if(!isDemand(msg.sender)) revert(); 
    Demands[msg.sender].Funding = Funding;
    emit
    LogUpdateDemand(
      msg.sender, 
      Demands[msg.sender].index,
      Funding, 
      Demands[msg.sender].maxPrice);
    return true;
  }
  
  function updatemaxPrice(uint maxPrice) 
    public
    returns(bool success) 
  {
    if(!isDemand(msg.sender)) revert(); 
    Demands[msg.sender].maxPrice = maxPrice;
    emit
    LogUpdateDemand(
      msg.sender, 
      Demands[msg.sender].index,
      Demands[msg.sender].Funding, 
      maxPrice);
    return true;
  }

  function getUserCount() 
    public
    constant
    returns(uint count)
  {
    return DemandIndex.length;
  }

  function getUserAtIndex(uint index)
    public
    constant
    returns(address userAddress)
  {
    return DemandIndex[index];
  }


  function getScore(address Owner)
    public
    constant
    returns(int score)
  {
    return Scores[Owner];
  }

  function allowRating(
    address    rater, 
    address    rated) 
    public
    returns(bool success)
  {
    canRate[rater][rated] = true;
    return true;
  }
  
  function Rate(
    address    rated,
    int       score)
    public
    returns(bool success)
  {
    if (!canRate[msg.sender][rated]) revert();
    if (score < -1) revert();
    if (score > 1) revert();
    Scores[rated] += score;
    canRate[msg.sender][rated] = false;
    emit 
        LogNewRate(
        rated, 
        score);
    return true;
  }  

}

'''

Propietarios_compiled_sol = compile_source(Propietarios_source_code,  import_remappings=['=./', '-']) # Compiled source code
Propietarios_interface = Propietarios_compiled_sol['<stdin>:Propietarios']

propietarios = w3.eth.contract(
    address = '0xc493Fef24cC0E42Db3627a703dC756958a2Fa104',
    abi = Propietarios_interface['abi'],
)

## delete demands

In [31]:
for i in range(1,len(public_keys)):
    address = public_keys[i]
    #acct = w3.eth.account.privateKeyToAccount(private_keys[i])
    if propietarios.call().isDemand(public_keys[i]) == True:
        acct = w3.eth.account.privateKeyToAccount(private_keys[i])
        construct_txn = propietarios.functions.deleteDemand(acct.address).buildTransaction({
            'from': acct.address,
            'nonce': w3.eth.getTransactionCount(acct.address),
            'gasPrice': w3.toWei('1', 'gwei')})
        signed = acct.signTransaction(construct_txn)
        tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
        print('Gas used by Delete Demand = '+str(tx_receipt.gasUsed))


Gas used by Delete Demand = 25331


BadFunctionCallOutput: Could not decode contract function call isDemand return data b'' for output_types ['bool']

## Proveedores

In [2]:
# Solidity source code
Proveedores_source_code = '''

pragma solidity ^0.4.24;

//import "./Owned.sol";

contract Proveedores{

  struct Provider {
    uint Price;
    uint index;
  }
  
  mapping(address => Provider) private Providers;
  address[] private ProviderIndex;

  mapping(address => int) private Scores;
  mapping(address => mapping(address => bool)) canRate;
  
  event LogNewProvider   (address indexed userAddress, uint index, uint maxPrice);
  event LogUpdateProvider(address indexed userAddress, uint index, uint maxPrice);
  event LogDeleteProvider(address indexed userAddress, uint index);
  event LogNewRate(address indexed Rated, int Score);
  
  function isProvider(address userAddress)
    public 
    constant
    returns(bool isIndeed) 
  {
    if(ProviderIndex.length == 0) return false;
    return (ProviderIndex[Providers[userAddress].index] == userAddress);
  }

  function newProvider(uint Price) 
    public
    returns(uint index)
  {
    if(isProvider(msg.sender)) revert(); 
    Providers[msg.sender].Price   = Price;
    Providers[msg.sender].index     = ProviderIndex.push(msg.sender)-1;
    emit 
        LogNewProvider(
        msg.sender, 
        Providers[msg.sender].index, 
        Price);
    return ProviderIndex.length-1;
  }

  function deleteProvider(address userAddress) 
    public
    returns(uint index)
  {
    if(!isProvider(userAddress)) revert(); 
    uint rowToDelete = Providers[userAddress].index;
    address keyToMove = ProviderIndex[ProviderIndex.length-1];
    ProviderIndex[rowToDelete] = keyToMove;
    Providers[keyToMove].index = rowToDelete; 
    ProviderIndex.length--;
    emit
    LogDeleteProvider(
        userAddress, 
        rowToDelete);
    emit
    LogUpdateProvider(
        keyToMove, 
        rowToDelete, 
        Providers[keyToMove].Price);
    return rowToDelete;
  }
  
  function getProvider(address userAddress)
    public 
    constant
    returns(uint maxPrice)
  {
    if(!isProvider(userAddress)) revert(); 
    return(
      Providers[userAddress].Price); 
      //Providers[userAddress].index);
  } 
  
  function updatePrice(uint newPrice) 
    public
    returns(bool success) 
  {
    if(!isProvider(msg.sender)) revert(); 
    Providers[msg.sender].Price = newPrice;
    emit
    LogUpdateProvider(
      msg.sender, 
      Providers[msg.sender].index,
      newPrice);
    return true;
  }

  function getProvidersCount() 
    public
    constant
    returns(uint count)
  {
    return ProviderIndex.length;
  }

  function getProviderAtIndex(uint index)
    public
    constant
    returns(address userAddress)
  {
    return ProviderIndex[index];
  }
  
  function getScore(address vendor)
    public
    constant
    returns(int score)
  {
    return Scores[vendor];
  }

  function allowRating(
    address    rater, 
    address    rated) 
    public
    returns(bool success)
  {
    canRate[rater][rated] = true;
    return true;
  }
  
  function Rate(
    address    rated,
    int       score)
    public
    returns(bool success)
  {
    if (!canRate[msg.sender][rated]) revert();
    if (score < -1) revert();
    if (score > 1) revert();
    Scores[rated] += score;
    canRate[msg.sender][rated] = false;
    emit 
        LogNewRate(
        rated, 
        score);
    return true;
  }   

}
'''


Proveedores_compiled_sol = compile_source(Proveedores_source_code,  import_remappings=['=./', '-']) # Compiled source code
Proveedores_interface = Proveedores_compiled_sol['<stdin>:Proveedores']

proveedores = w3.eth.contract(
    address = '0x88679873522CEEADC39A00187634B657625caF0e',
    abi = Proveedores_interface['abi'],
)

## clear demands

In [8]:
for i in range(1,len(public_keys)):
    #address = public_keys[i]
    if proveedores.call().isProvider(public_keys[i]) == True:
        acct = w3.eth.account.privateKeyToAccount(private_keys[i])
        construct_txn = proveedores.functions.deleteProvider(acct.address).buildTransaction({
            'from': acct.address,
            'nonce': w3.eth.getTransactionCount(acct.address),
            'gasPrice': w3.toWei('1', 'gwei')})
        signed = acct.signTransaction(construct_txn)
        tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
        tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
        print('Gas used by Delete Demand = '+str(tx_receipt.gasUsed))


Gas used by Delete Demand = 25062
